In [1]:
import json
import requests
import pandas as pd

In [2]:
r = requests.get("http://berkeleyearth.lbl.gov/air-quality/maps/cities/United_States_of_America/Louisiana/Baton_Rouge.txt")


In [3]:
import urllib.request  as urllib2
import datetime
#data = urllib2.urlopen("http://berkeleyearth.lbl.gov/air-quality/maps/cities/Sweden/Stockholm/Stockholm.txt") # it's a file like object and works just like a file
data = urllib2.urlopen("http://berkeleyearth.lbl.gov/air-quality/maps/cities/United_States_of_America/California/Fresno.txt") # it's a file like object and works just like a file
#data = urllib2.urlopen("http://berkeleyearth.lbl.gov/air-quality/maps/cities/India/NCT/New_Delhi.txt") # it's a file like object and works just like a file

City = []
dtlist = []
PMList = []

for line in data:
    
    formattedLine = line.decode("utf-8")
    if "City:" in formattedLine:
        City = formattedLine.split(':')[1]
        continue
    elif formattedLine.startswith('%'):
        continue
    else:
        setOfTokens = formattedLine.split()
    #print(setOfTokens)
    t = datetime.time(int(setOfTokens[3]))
    d = datetime.date(int(setOfTokens[0]), int(setOfTokens[1]), int(setOfTokens[2]))
    dt = datetime.datetime.combine(d,t)
    
    dtlist.append(dt)
    
    PM = setOfTokens[4]
    PMList.append(PM)
       

In [4]:
#import pandas as pd


#data = pd.read_csv('http://berkeleyearth.lbl.gov/air-quality/maps/cities/United_States_of_America/Louisiana/Baton_Rouge.txt',sep = '\t', header=None)
print(len(dtlist),len(PMList))
d={'date':dtlist,'PM':PMList}

145770 145770


In [5]:
df = pd.DataFrame(d, columns=['date','PM'])
print(type(df.date[0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [6]:
df = df.set_index('date')

In [7]:
mask = (df.index >= '2018-01-01') & (df.index <= '2018-12-31')
df= df.loc[mask]

In [8]:
import numpy as np
#data = pd.read_csv('C:/Users/charl/Desktop/bigdaat/Stockholm_Temp_2016.csv',sep = ',', header=None)
data = pd.read_csv('C:/Users/charl/Desktop/bigdaat/Fresno_Dew_2016.csv',sep = ',', header=None)
#data = pd.read_csv('C:/Users/charl/Desktop/bigdaat/New_Delhi_Temp_2016.csv',sep = ',', header=None)
df1 =pd.DataFrame(data)
df1.columns = ['Source', 'Station', 'Date', 'zdf', 'Hour', 'Temp']
from datetime import datetime

df1['Date'] =df1['Date'] + " " + df1['Hour']+":00"

df1['Date'] = pd.to_datetime(df1['Date'])

del df1['Source']
del df1['Station']
del df1['Hour']
del df1['zdf']

#df1 = df1.set_index('Date')

df1= df1.replace('\'-', np.nan)

df1['Temp']=df1['Temp'].astype(float)
#df1['Dew']=df1['Dew'].astype(float)#
#df1 = df1.loc[~df1.index.duplicated(keep='last')]

In [9]:
import datetime as dt
print(type(df1.index[0]))

#df1 = df1.loc[~df1.index.duplicated(keep='last')]
#df1 = df1.assign(Date=df1.Date.dt.round('H')).head(10)


df1 = df1.assign(Date=df1.Date.dt.round('H'))
df1 = df1.set_index('Date')



<class 'int'>


In [10]:
mask = (df1.index >= '2018-01-01') & (df1.index <= '2018-12-31')
df1 = df1.loc[mask]

In [11]:
print(df1.tail(8))

                     Temp
Date                     
2018-12-30 00:00:00   NaN
2018-12-30 01:00:00   NaN
2018-12-30 02:00:00   NaN
2018-12-30 03:00:00   NaN
2018-12-30 04:00:00   NaN
2018-12-30 05:00:00   NaN
2018-12-30 06:00:00   NaN
2018-12-30 07:00:00   NaN


In [12]:
#bigdata = df1.append(df, ignore_index=True)
merge=pd.merge(df1,df, how='inner', left_index=True, right_index=True)
dfmerge = merge.reset_index()
dfmerge = dfmerge.dropna(how='any')
dfmerge.columns = ['datetime', 'Temp','PM']
dfmerge = dfmerge.astype({"datetime": str})
dfmerge['Tempmean'] = dfmerge.apply (lambda row: row[1] - avg1, axis=1)
dfmerge['PMmean'] = dfmerge.apply (lambda row: float(row[2]) - avg2, axis=1)
dfmerge['Numerator'] = dfmerge.apply (lambda row: row[3]*float(row[4]), axis=1)
dfmerge['Tempmean2'] = dfmerge.apply (lambda row: row[3]*row[3], axis=1)
dfmerge['PMmean2'] = dfmerge.apply (lambda row: float(row[4])*float(row[4]), axis=1)


NameError: ("name 'avg1' is not defined", 'occurred at index 0')

In [ ]:
print(dfmerge.head(9))

In [ ]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

In [ ]:
#from pyspark import SparkContext, SparkConf
#from pyspark.sql import SparkSession
#conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
#sc = pyspark.SparkContext(conf=conf)
#spark = SparkSession(sc)

In [ ]:
sc.stop()
import findspark
findspark.init()
import pyspark # Call this only after findspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
spark_df = spark.createDataFrame(dfmerge)

In [ ]:

#mask = (dfmerge.index >= '2016-01-01') & (dfmerge.index <= '2016-12-31')
#dfmerge.loc[mask]

In [ ]:
spark_df.take(10)

In [ ]:
rdd = spark_df.rdd.map(tuple)

In [ ]:
rdd.take(10)

In [ ]:
a = rdd.map(lambda a: float(a[3])*float(a[4]))

In [ ]:
a.take(10)

In [ ]:
avg1 = rdd.map(lambda x: x[1]).reduce(lambda x, b: x+b)/a.count()

In [ ]:
print(avg1)

In [ ]:
avg2 = rdd.map(lambda x: float(x[2])).reduce(lambda x, b: x+b)/a.count()
print(avg2)

avg2 = rdd.map(lambda x: float(x[2])).reduce(lambda x, b: x+b)/a.count()
avg2

In [ ]:
rdd.take(5)

In [ ]:
num = rdd.map(lambda x: x[5]).reduce(lambda x, b: x+b)
print(num, a.count())

In [ ]:
import math
x2sum = rdd.map(lambda x: x[6]).reduce(lambda x, b: x+b)
y2sum = rdd.map(lambda x: x[7]).reduce(lambda x, b: x+b)
print(x2sum, y2sum)
num/(math.sqrt(x2sum * y2sum))

In [ ]:
yterm = rdd.map(lambda x: float(x[2])).reduce(lambda x, b: x+b) - a.count()*(10)
print(yterm, a.count())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(20, 4)})

x_values1=df1.index
y_values1=df1['Temp']

x_values2=df.index
y_values2=df['PM']


fig=plt.figure()
ax=fig.add_subplot(111, label="1")
ax2=fig.add_subplot(111, label="2", frame_on=False)

ax.plot(x_values1, y_values1, color="C0")
ax.set_xlabel("Date", color="C0")
ax.set_ylabel("Dew in F", color="C0")
ax.tick_params(axis='x', colors="C0")
ax.tick_params(axis='y', colors="C0")

ax2.scatter(x_values2, y_values2, color="C1")
ax2.xaxis.tick_top()
ax2.yaxis.tick_right()
ax2.set_xlabel('Date', color="C1") 
ax2.set_ylabel('PM2.5', color="C1")       
ax2.xaxis.set_label_position('top') 
ax2.yaxis.set_label_position('right') 
ax2.tick_params(axis='x', colors="C1")
ax2.tick_params(axis='y', colors="C1")

#ax3.plot(x_values3, y_values3, color="C3")
#ax3.set_xticks([])
#ax3.set_yticks([])

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
x1 = df1.index
y1 = df1['Temp']
x2 = df.index
y2 = df['PM']

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()

#ax1.plot(x1, y1, 'g-')
#ax2.plot(x2, y2, 'b-')

ax1.set_xlabel('X data')
ax1.set_ylabel('Temp data', color='g')
ax2.set_ylabel('PM2.5 data', color='b')

#plt.scatter(x1,y1)
#plt.scatter(x2,y2)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(20, 4)})

#df1['Temp'].plot(linewidth=0.5)
df1['Temp'].plot(linewidth=0.5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(20, 4)})

df['PM']=df['PM'].astype(float)

#df1['Temp'].plot(linewidth=0.5)
df['PM'].plot(linewidth=0.5)

In [ ]:
print(df.set_index('date'))
print(df.head(9))